In [4]:
import datetime
import time
import requests
import pandas as pd
from sqlalchemy import create_engine

# Archive.org crawling

In [ ]:
#https://stackoverflow.com/questions/28154066/how-to-convert-datetime-to-integer-in-python
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [ ]:
#https://stackoverflow.com/questions/993358/creating-a-range-of-dates-in-python
start = datetime.datetime(2017, 12, 30)
dates = [start - datetime.timedelta(days=x) for x in range(0, 1500)]

In [ ]:
intdates = [to_integer(date) for date in dates]

In [ ]:
not_testing = []
for i in intdates:
    r = requests.get('http://archive.org/wayback/available?url=nytimes.com&timestamp={}'.format(str(i)))
    arc = r.json()['archived_snapshots']['closest']['url']
    r2 = requests.get(arc)
    if 'optimizely.com' not in r2.text:
        print(arc)
        not_testing.append(arc)
    else:
        print(i)
        time.sleep(2)

In [ ]:
#http://web.archive.org/web/20150805230843/http://www.nytimes.com:80/

# NYT headlines

Format: http://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={your-api-key}

In [52]:
key = %env KEY

def query_range(startmo, startyr, endmo, endyr):
    startmos = [[i, startyr] for i in range(startmo, 13)]
    middle = [[j, i] for i in range(startyr, endyr+1) if i != startyr and i != endyr for j in range(1, 13)]
    endmos = [[i, endyr] for i in range(1, endmo+1)]
    allmos = [i for sub in [startmos, middle, endmos] for i in sub]
    return allmos

def cleaning(entry):
    try:
        entry['print_headline'] = entry['headline']['print_headline']
    except Exception:
        pass
    try:
        entry['headline'] = entry['headline']['main']
    except Exception:
        entry['headline'] = str(entry['headline'])
    try:
        entry['byline'] = str(entry['byline'])
    except Exception:
        entry['byline'] = ''
    entry['keywords'] = ','.join(i['value'] for i in entry['keywords'])
    if 'new_desk' in entry:
        entry['news_desk'] = entry['new_desk']
        entry.pop('new_desk', None)
    entry.pop('multimedia', None)
    entry.pop('blog', None)
    entry.pop('score', None)
    entry.pop('uri', None)
    return entry

In [53]:
morange = query_range(2, 2017, 9, 2018)

for i in morange:
    print(i)
    h = requests.get('http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'.format(i[1], i[0], key))
    items = h.json()['response']['docs']
    cleaned = [cleaning(i) for i in items]
    cdf = pd.DataFrame(cleaned)
    cdf.to_sql('results', engine, if_exists = 'append')

[2, 2017]
[3, 2017]
[4, 2017]
[5, 2017]
[6, 2017]
[7, 2017]
[8, 2017]
[9, 2017]
[10, 2017]
[11, 2017]
[12, 2017]
[1, 2018]
[2, 2018]
[3, 2018]
[4, 2018]
[5, 2018]
[6, 2018]
[7, 2018]
[8, 2018]
[9, 2018]


In [39]:
h = requests.get('http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'.format(2018, 5, key))
items = h.json()['response']['docs']
cleaned = [cleaning(i) for i in items]
cleaned

[{'web_url': 'https://www.nytimes.com/2018/05/02/briefing/israel-mike-pompeo-liverpool.html',
  'snippet': 'Here’s what you need to know to start your day.',
  'source': 'The New York Times',
  'headline': 'Israel, Mike Pompeo, Liverpool: Your Thursday Briefing',
  'keywords': '',
  'pub_date': '2018-05-02T19:58:56+0000',
  'document_type': 'article',
  'byline': "{'original': 'By CHARLES McDERMID', 'person': [{'firstname': 'Charles', 'middlename': None, 'lastname': 'McDERMID', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}], 'organization': None}",
  'type_of_material': 'briefing',
  '_id': '5aea1882068401528a2abe87',
  'word_count': 1292,
  'score': 1,
  'uri': 'nyt://article/cf9dbb5d-8a96-534d-8abb-a7dce9014521',
  'print_headline': '',
  'news_desk': 'NYTNow'},
 {'web_url': 'https://www.nytimes.com/2018/05/02/health/e-coli-romaine-update.html',
  'snippet': 'It is the first reported death in the outbreak, which has caused 121 reported infection

In [19]:
user = %env USER
password = %env PASSWORD
db = %env DATABASE
engine = create_engine('postgresql://{}:{}@localhost:5432/{}'.format(user, password, db))

cdf.to_sql('test', engine)

In [ ]:
cdf['keyword1'].dtype

In [40]:
cdf = pd.DataFrame(cleaned)
cdf

,_id,byline,document_type,headline,keywords,news_desk,print_headline,print_page,pub_date,score,section_name,slideshow_credits,snippet,source,type_of_material,uri,web_url,word_count
0,5aea1882068401528a2abe87,"{'original': 'By CHARLES McDERMID', 'person': ...",article,"Israel, Mike Pompeo, Liverpool: Your Thursday ...",,NYTNow,,NaN,2018-05-02T19:58:56+0000,1,NaN,NaN,Here’s what you need to know to start your day.,The New York Times,briefing,nyt://article/cf9dbb5d-8a96-534d-8abb-a7dce901...,https://www.nytimes.com/2018/05/02/briefing/is...,1292
1,5aea3ef1068401528a2abeef,"{'original': 'By MAGGIE ASTOR', 'person': [{'f...",article,E. Coli Outbreak Turns Deadly With a Fatal Cas...,"E Coli (Bacteria),Food Contamination and Poiso...",National,"E. Coli Outbreak Spreads to 25 States, and the...",14,2018-05-02T22:42:51+0000,1,NaN,NaN,It is the first reported death in the outbreak...,The New York Times,News,nyt://article/3c4f5610-9095-5ff3-adb7-1d3b5086...,https://www.nytimes.com/2018/05/02/health/e-co...,345
2,5ae9d04f068401528a2abd76,"{'original': 'By TYLER KEPNER', 'person': [{'f...",article,Matt Harvey Comes to a Crossroads With the Mets,"Baseball,Harvey, Matt,New York Mets",Sports,Chance to Start Again Begins by Working Out of...,11,2018-05-02T14:50:53+0000,1,Baseball,NaN,Harvey is now in the bullpen. With Jacob deGro...,The New York Times,News,nyt://article/1879bb91-96c4-5458-a16c-4f94adbb...,https://www.nytimes.com/2018/05/02/sports/base...,1200
3,5ae9e08c068401528a2abdb1,"{'original': 'By NELLIE BOWLES', 'person': [{'...",article,All We Want to Do Is Watch Each Other Play Vid...,"Fortnite (Video Game),Computer and Video Games...",Styles,Generation Game On,1,2018-05-02T16:00:07+0000,1,NaN,NaN,Gamers are the new stars. Esports arenas are t...,The New York Times,News,nyt://article/cb09d90a-65d4-5885-8abf-5387bd30...,https://www.nytimes.com/2018/05/02/style/fortn...,1858
4,5ae945db068401528a2abc05,"{'original': 'By MAYA GOODFELLOW', 'person': [...",article,A New Face Won’t Change the British Government...,"Great Britain,Javid, Sajid (1969- ),May, There...",OpEd,The British Government’s Racist Heart,23,2018-05-02T05:00:05+0000,1,NaN,NaN,Theresa May’s immigration policy needs more th...,The New York Times,Op-Ed,nyt://article/ad0d4a7e-6a38-583c-8a6d-3a63c428...,https://www.nytimes.com/2018/05/02/opinion/saj...,860
5,5ae9f93a068401528a2abe1f,"{'original': 'By JULIET MACUR', 'person': [{'f...",article,Washington Redskins Cheerleaders Describe Topl...,"Football,Cheerleaders,Washington Redskins,Sexu...",Sports,"For Cheerleaders, a Trip Had Unwelcome Guests",1,2018-05-02T17:45:23+0000,1,NaN,NaN,Men were granted up-close access to photo shoo...,The New York Times,News,nyt://article/a0906e41-61eb-5a18-ab3c-a54368b3...,https://www.nytimes.com/2018/05/02/sports/reds...,2559
6,5aea114d068401528a2abe71,"{'original': 'By JESSE McKINLEY', 'person': [{...",article,"Elected on G.O.P. Line, a Democrat Sits Alone","Bohen, Erik T,New York State,Politics and Gove...",Metro,"Elected on G.O.P. Line, a Democrat Is Banished...",20,2018-05-02T19:28:11+0000,1,NaN,NaN,"After arriving in the New York State Assembly,...",The New York Times,News,nyt://article/27d40040-f5fe-57ae-af4b-36cf2aba...,https://www.nytimes.com/2018/05/02/nyregion/as...,635
7,5ae9e9b6068401528a2abde8,"{'original': 'By PARUL SEHGAL', 'person': [{'f...",article,‘Barracoon’ and ‘Slave Old Man’ Approach the T...,"Books and Literature,Hurston, Zora Neale,Black...",Culture,A Trauma Approached With Care and Kinship,4,2018-05-02T16:39:15+0000,1,NaN,NaN,Patrick Chamoiseau and Zora Neale Hurston tell...,The New York Times,Review,nyt://article/3ba1c452-2551-5d19-9e57-dbe94b4d...,https://www.nytimes.com/2018/05/02/books/revie...,1109
8,5ae9d427068401528a2abd8b,"{'original': 'By AMY QIN', 'person': [{'firstn...",article,Teenager’s Prom Dress Stirs Furor in U.S. — bu...,"Proms,Fashion and Apparel,Asian-Americans,Chin...",Foreign,Furor Over a Prom Dress in America Baffles Chi...,8,2018-05-02T15:07:15+0000

In [44]:
cdf

,_id,abstract,byline,document_type,headline,keywords,lead_paragraph,news_desk,print_headline,print_page,pub_date,section_name,slideshow_credits,snippet,source,subsection_name,type_of_material,web_url,word_count
0,52a1070c38f0d84e53acb910,None,None,multimedia,"Mandela as Dissident, Liberator and Statesman","Apartheid (Policy),Race and Ethnicity,Deaths (...","Nelson Mandela, the leading emancipator of Sou...",World / Africa,NaN,None,2013-01-01T00:00:00Z,World,None,"Nelson Mandela, the leading emancipator of Sou...",The New York Times,Africa,Interactive Feature,https://www.nytimes.com/interactive/2013/01/01...,0
1,50e3a56900315214fbb81fdf,David Steinberg returns to artificially color ...,"{'person': [{'organization': '', 'role': 'repo...",blogpost,Making Scents of Things,Crossword Puzzles,None,None,NaN,None,2013-01-01T22:00:26Z,Crosswords/Games,None,David Steinberg returns to artificially color ...,The New York Times,None,Blog,https://wordplay.blogs.nytimes.com/2013/01/01/...,365
2,513e021fcf28d04ae0001018,None,[],multimedia,Passage of Tax Deal Hangs in the Balance,"United States Politics and Government,Biden, J...","At the end of the 112th Congress, lawmakers lu...",U.S.,NaN,None,2013-01-01T00:00:00Z,U.S.,None,"At the end of the 112th Congress, lawmakers lu...",The New York Times,None,Slideshow,https://www.nytimes.com/slideshow/2013/01/01/u...,21
3,50e39db900315214fbb81fd4,A 405-year-old thought from Cervantes applies ...,"{'person': [{'firstname': 'Andrew', 'middlenam...",blogpost,Advice from Cervantes that Suits 2013 and Beyo...,"Books and Literature,Forecasts",None,None,NaN,None,2013-01-01T21:22:53Z,Opinion,None,A 405-year-old thought from Cervantes applies ...,The New York Times,None,Blog,https://dotearth.blogs.nytimes.com/2013/01/01/...,342
4,50e3929900315214fbb81fc3,None,[],article,Holiday on Tuesday: New Year’s Day,Holidays and Special Occasions,A list of government offices and other service...,Metro,Holiday on Tuesday: New Year’s Day,17,2013-01-01T00:00:00Z,N.Y. / Region,None,A list of government offices and other service...,The New York Times,None,List,https://www.nytimes.com/2013/01/01/nyregion/ho...,184
5,50e38d6e00315214fbb81fb8,After a year of legal wrangling between the de...,"{'person': [{'organization': '', 'role': 'repo...",blogpost,Agreement Announced in Tory Burch Case,"Burch, Chris,Burch, Tory,C Wonder,Fashion and ...",None,None,Tory Burch Lawsuits Are Settled,None,2013-01-01T20:21:26Z,Style,None,After a year of legal wrangling between the de...,The New York Times,Fashion &amp; Style,Blog,https://runway.blogs.nytimes.com/2013/01/01/ag...,247
6,50e387dd00315214fbb81fb2,Let's resolve to develop a clear plan for impr...,"{'person': [{'organization': '', 'role': 'repo...",blogpost,Fixing Our Food Problem,"Agriculture and Farming,Factory Farming,Food,O...",None,None,Fixing Our Food Problem,None,2013-01-01T19:52:19Z,Opinion,None,Let's resolve to develop a clear plan for impr...,The New York Times,None,Blog,https://opinionator.blogs.nytimes.com/2013/01/...,786
7,513e05b0cf28d04ae0001eee,None,[],multimedia,‘Maria Stuarda’ at the Metropolitan Opera,"Opera,Metropolitan Opera,DiDonato, Joyce,van d...",Images from the production.,Arts / Music,NaN,None,2013-01-01T00:00:00Z,Arts,None,Images from the production....,The New York Times,Music,Slideshow,https://www.nytimes.com/slideshow/2013/01/01/a...,4
8,50e37ab500315214fbb81f94,"After its Broadway run finishes, ""Peter and th...","{'person': [{'organization': '', 'role': 'repo...",blogpost,'Peter and the Starcatcher' Headed Off Broadway,"New World Stages,Theater",None,None,‘Peter and the Starcatcher’ Headed Off Broadway,None,2013-01-01T19:00:33Z,Arts,None,"After its Broadway run finishes, ""Peter and th...",The New York Times,None,Blog,https://artsbeat.blogs.nytimes.com/2013/01/01/...,209
9,50e376cb00315214fbb81f8e,"Amar'e Stoudemire, who had knee surgery in Oct...","{'person': [{'organization': '', 'role': 'repo...",blogpost,Stoudemire to Come Off the Bench Against Portland